<a href="https://colab.research.google.com/github/Ireneyou33/Model-Comparison-and-Forecasting-on-COVID-19/blob/master/NN_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout
import datetime as dt
from tensorflow.python.keras.layers import Dense, LSTM
from tensorflow.python.keras import Sequential
from keras.models import load_model
from keras.models import model_from_json
from datetime import datetime, timedelta
import json
import math

In [ ]:
%run functions.ipynb

In [ ]:
def NeuralNets(train=True, world=True, country=None, w=1):
    if world:
        dataframe = worldclass_dataframe(df_conf)
    else:
        dataframe, _, _ = sum_regioncase(df_conf, country, region_level="county")
    X_train, Y_train, X_test, Y_test = get_traintest(dataframe, idx, w=w, r=1)
        
    if train:
        model = Sequential()
        model.add(Dense(8, input_dim=w, activation='relu'))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        model.fit(X_train, Y_train, epochs=200, batch_size=2, verbose=0)
        print("With window size {}".format(w))
        trainScore = model.evaluate(X_train, Y_train, verbose=0)
        print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, math.sqrt(trainScore)))
        testScore = model.evaluate(X_test, Y_test, verbose=0)
        print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))
        model.save('worldNN_w{}.h5'.format(w))
    else:
        model = tf.keras.models.load_model('worldNN_w{}.h5'.format(w))
#         model.fit(X_train, Y_train, epochs=200, batch_size=2, verbose=0)
        print("Load Model")
        print("With window size {}".format(w))
        trainScore = model.evaluate(X_train, Y_train, verbose=0)
        print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, math.sqrt(trainScore)))
        testScore = model.evaluate(X_test, Y_test, verbose=0)
        print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))
    return model

In [ ]:
def NN_pred(model, world=True, country=None, w=1):
    if world:
        dataframe = worldclass_dataframe(df_conf)
    else:
        dataframe, _, _ = sum_regioncase(df_conf, country, region_level="county")
    X_train, Y_train, X_test, Y_test = get_traintest(dataframe, idx, w=w, r=1)
    
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)
    pred = np.concatenate((pred_train,pred_test),axis=0)
    nan = np.array([[np.nan]])
    a = np.concatenate((pred_train, nan))
    for i in range(w-1):
        a = np.concatenate((a, nan))
    b = np.concatenate((a,pred_test))
    c = np.concatenate((nan,b))
    for i in range(w-1):
        c = np.concatenate((nan,c))
    pred = c.reshape(-1)
    df_pred =pd.DataFrame([pred],columns=date)
    df = pd.concat([dataframe, df_pred])
    df.index=["Original Data", "Predicted Data"]
    return df

In [ ]:
def wfv(windowsize, dataframe, region="world", train=False, init_trainsize=30):
    train_RMSE=[]
    test_RMSE=[]
    for w in windowsize:
        if train:
            print("training with w = {}".format(w))
            tr_rmse, te_rmse, train_size = walk_forward_vali(dataframe, w=w, idx=init_trainsize)
            data=[tr_rmse, te_rmse, train_size]
            train_RMSE.append(tr_rmse)
            test_RMSE.append(te_rmse)
            with open("{}NN_W{}_mfv.json".format(region, w), 'w') as file:
                json.dump(data, file, indent=2) 
        else:
            with open("{}NN_W{}_mfv.json".format(region, w), 'r') as file:
                data = json.load(file)
            train_RMSE.append(data[0])
            test_RMSE.append(data[1])
            train_size=data[2]
    return train_RMSE, test_RMSE, train_size

LSTM

In [ ]:
def reshape_func(x, shape = 1, r=1):
    x = np.array(x)
    x = x.astype(int)
    X = x.reshape((x.shape[0], shape, r))
    return X

In [ ]:
def LSTM_model(train=True, world=True, country=None, w=1):
    if world:
        dataframe = worldclass_dataframe(df_conf)
    else:
        dataframe, _, _ = sum_regioncase(df_conf, country, region_level="county")
    X_train, Y_train, X_test, Y_test = get_traintest(dataframe, idx, w=w, r=1)
    if w ==1:
        X_train = reshape_func(X_train, r=1)
        X_test = reshape_func(X_test, r=1)
    else:
        X_train = reshape_func(X_train,shape=w, r=1)
        X_test = reshape_func(X_test, shape=w, r=1)
        
    if train:
        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(w, r)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        model.fit(X_train, Y_train, epochs=200, verbose=0)
        print("With window size {}".format(w))
        trainScore = model.evaluate(X_train, Y_train, verbose=0)
        print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, math.sqrt(trainScore)))
        testScore = model.evaluate(X_test, Y_test, verbose=0)
        print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))
        model.save('worldLSTM_w{}.h5'.format(w))
    else:
        model = tf.keras.models.load_model('worldLSTM_w{}.h5'.format(w))
        print("Load Model")
        print("With window size {}".format(w))
        trainScore = model.evaluate(X_train, Y_train, verbose=0)
        print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, math.sqrt(trainScore)))
        testScore = model.evaluate(X_test, Y_test, verbose=0)
        print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))
    return model

In [ ]:
def LSTM_pred(model, world=True, country=None, w=1):
    if world:
        dataframe = worldclass_dataframe(df_conf)
    else:
        dataframe, _, _ = sum_regioncase(df_conf, country, region_level="county")
    X_train, Y_train, X_test, Y_test = get_traintest(dataframe, idx, w=w, r=1)
    if w ==1:
        X_train = reshape_func(X_train, r=1)
        X_test = reshape_func(X_test, r=1)
    else:
        X_train = reshape_func(X_train,shape=w, r=1)
        X_test = reshape_func(X_test,shape=w, r=1)
    
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)
    pred = np.concatenate((pred_train,pred_test),axis=0)
    nan = np.array([[np.nan]])
    a = np.concatenate((pred_train, nan))
    for i in range(w-1):
        a = np.concatenate((a, nan))
    b = np.concatenate((a,pred_test))
    c = np.concatenate((nan,b))
    for i in range(w-1):
        c = np.concatenate((nan,c))
    pred = c.reshape(-1)
    df_pred =pd.DataFrame([pred],columns=date)
    df = pd.concat([dataframe, df_pred])
    df.index=["Original Data", "Predicted Data"]
    return df

In [ ]:
def wfv_lstm(windowsize, dataframe, region="world", train=False, init_trainsize=30):
    train_RMSE=[]
    test_RMSE=[]
    for w in windowsize:
        if train:
            print("training with w = {}".format(w))
            tr_rmse, te_rmse, train_size = walk_forward_vali_lstm(dataframe, w=w, idx=init_trainsize)
            data=[tr_rmse, te_rmse, train_size]
            train_RMSE.append(tr_rmse)
            test_RMSE.append(te_rmse)
            with open("{}LSTM_W{}_mfv.json".format(region, w), 'w') as file:
                json.dump(data, file, indent=2) 
        else:
            with open("{}LSTM_W{}_mfv.json".format(region, w), 'r') as file:
                data = json.load(file)
            train_RMSE.append(data[0])
            test_RMSE.append(data[1])
            train_size=data[2]
    return train_RMSE, test_RMSE, train_size

NeuralNets Forecast

In [ ]:
def NN_forecast(df, model, forecast_days=10, w=3, r=1):
    df_test = df.copy()
    df_test
    for i in range(forecast_days):
        a = df_test.columns[-1] + timedelta(days=1)
        df_test[a] = np.nan
        df_test[a].iloc[0] = df_test.iloc[1,-2]
        X_train, Y_train, X_test, Y_test = get_traintest(df_test, idx, w=w, r=r)
        y_pred = model.predict(X_test, verbose=0)
        if y_pred[-1] < df_test.iloc[-1,-2]: # Cumulative cases cannot decease
            y_pred[-1] = df_test.iloc[-1,-2]
        df_test.iloc[-1,-1]=y_pred[-1]
    return df_test

LSTM Forecast

In [1]:
def LSTM_forecast(df, model, forecast_days=10, w=3, r=1):
  df_test = df.copy()
  df_test
  for i in range(forecast_days):
    a = df_test.columns[-1] + timedelta(days=1)
    df_test[a] = np.nan
    df_test[a].iloc[0] = df_test.iloc[1,-2]
    X_tr, Y_tr, X_te, Y_te = get_traintest(df_test, idx, w=w, r=1)
    if w ==1:
      X_tr = reshape_func(X_tr, r=1)
      X_te = reshape_func(X_te, r=1)
    else:
      X_tr = reshape_func(X_tr,shape=w, r=1)
      X_te = reshape_func(X_te, shape=w, r=1)
    
    y_pred = model.predict(X_te, verbose=0)
    if y_pred[-1] < df_test.iloc[-1,-2]: # Cumulative cases cannot decease
      y_pred[-1] = df_test.iloc[-1,-2]
    df_test.iloc[-1,-1]=y_pred[-1]
  return df_test